## Part1:File Structure

- Give your project a proper file structure.  There should be at least one (but maybe more!) folders.  Some possible folders:
Data or dat
Scripts or scr
Documents or doc
Results
Clean_Data
- Think about what should live in your root:  at the minimum, LICENCE, README, .gitignore
- If your data is in a different folder than your scripts, you may need to use pd.read_csv("../data/data.csv") as your relative path.
- You will be using only relative paths, not absolute paths.  This means that anyone else that forks your project can run everything, without having to change the path.
- Don't merge in your branch until I've released grades!
- Take a screen grab of the local version of your project, as the data is in the .gitignore file.

## Part2:Joins

- Find another dataset that will join with your original data.  This can be anything that you would like, as long as there is a key in common.
- If you cannot find any real data, you may wish to generate some fake data - Just cite that.

In [5]:
import pandas as pd

# Load the datasets
df_diabetes = pd.read_csv("../data/diabetes.csv")
df_hospital = pd.read_csv("../data/diabetic_data.csv")

# Display the first few rows to understand the structure
print(df_diabetes.head(3))
#print(df_hospital.head(3))


   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   

   DiabetesPedigreeFunction  Age  Outcome  
0                     0.627   50        1  
1                     0.351   31        0  
2                     0.672   32        1  


In [6]:
# Define age bins and labels to match `diabetic_data.csv`
def convert_age_to_range(age):
    if age < 10:
        return "[0-10)"
    elif age < 20:
        return "[10-20)"
    elif age < 30:
        return "[20-30)"
    elif age < 40:
        return "[30-40)"
    elif age < 50:
        return "[40-50)"
    elif age < 60:
        return "[50-60)"
    elif age < 70:
        return "[60-70)"
    elif age < 80:
        return "[70-80)"
    elif age < 90:
        return "[80-90)"
    else:
        return "[90-100)"

# Apply the function to the `Age` column
df_diabetes["age"] = df_diabetes["Age"].apply(convert_age_to_range)

# Drop the old `Age` column (optional)
df_diabetes = df_diabetes.drop(columns=["Age"])

# Display updated dataframe
print(df_diabetes.head())


   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   
3            1       89             66             23       94  28.1   
4            0      137             40             35      168  43.1   

   DiabetesPedigreeFunction  Outcome      age  
0                     0.627        1  [50-60)  
1                     0.351        0  [30-40)  
2                     0.672        1  [30-40)  
3                     0.167        0  [20-30)  
4                     2.288        1  [30-40)  


In [7]:
print(df_hospital.dtypes)
print(df_hospital[["time_in_hospital", "num_lab_procedures", "num_medications", "readmitted"]].head(10))


encounter_id                 int64
patient_nbr                  int64
race                        object
gender                      object
age                         object
weight                      object
admission_type_id            int64
discharge_disposition_id     int64
admission_source_id          int64
time_in_hospital             int64
payer_code                  object
medical_specialty           object
num_lab_procedures           int64
num_procedures               int64
num_medications              int64
number_outpatient            int64
number_emergency             int64
number_inpatient             int64
diag_1                      object
diag_2                      object
diag_3                      object
number_diagnoses             int64
max_glu_serum               object
A1Cresult                   object
metformin                   object
repaglinide                 object
nateglinide                 object
chlorpropamide              object
glimepiride         

In [8]:
df_hospital["readmitted"] = df_hospital["readmitted"].map({"NO": 0, "<30": 1, ">30": 1})
# This way, readmitted represents whether the patient was readmitted (1) or not (0).

In [9]:
df_hospital[["time_in_hospital", "num_lab_procedures", "num_medications"]] = df_hospital[["time_in_hospital", "num_lab_procedures", "num_medications"]].apply(pd.to_numeric, errors="coerce")
# This forces non-numeric values to NaN, which will be ignored in aggregation.

In [10]:
# Aggregate hospital data to get average values for each age group
df_hospital_grouped = df_hospital.groupby("age").agg({
    "time_in_hospital": "mean",
    "num_lab_procedures": "mean",
    "num_medications": "mean",
    "readmitted": "mean"  # Proportion of readmitted patients
}).reset_index()

# Merge the two datasets
merged_df = pd.merge(df_diabetes, df_hospital_grouped, on="age", how="inner")

print(f"Original diabetes dataset size: {df_diabetes.shape}")
print(f"Original hospital dataset size: {df_hospital.shape}")
print(f"Merged dataset size: {merged_df.shape}")

Original diabetes dataset size: (768, 9)
Original hospital dataset size: (101766, 50)
Merged dataset size: (768, 13)


### My explaination : Improved Predictive Power is the reason why I merge these two datasets.

Now with a merged dataset that combines medical history with hospital readmission records. 
Means that I am being able to use this for predictive modeling to determine which diabetic patient group are at higher risk of hospital readmission.

### Why Use inner Instead of Other Options?
An inner merge keeps only the rows where the age values exist in both datasets, since I dont want to make the dataset way much larger then the original datasets. What I would like to see is that if an age exists in "df_diabetes" but not in "df_hospital_grouped", it is dropped.
Or if I choose to do the left or other merge types, there could be 'NaN' values in columns where no match was found.
Since my goal is to analyze only the age groups present in both datasets, an inner merge makes sense. 